In [1]:
versioninfo()

Julia Version 0.5.0
Commit 3c9d753 (2016-09-19 18:14 UTC)
Platform Info:
  System: Darwin (x86_64-apple-darwin13.4.0)
  CPU: Intel(R) Core(TM) i7-4870HQ CPU @ 2.50GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.7.1 (ORCJIT, haswell)


In [2]:
using OpenCL
using Gadfly

In [3]:
diff_kernel = "
inline int xy_to_i(int x, int y, int col_size, int mesh_size) {
    return (x % col_size) + (y % col_size) * col_size; 
}

__kernel void diff(__global const float *a,
__global float *diff,
int col_size,
int mesh_size)
    {
      int gid = get_global_id(0);
    int x_idx = gid % col_size;
    int y_idx = gid / col_size;
    
    diff[gid] = - 4. * a[gid]
    + a[xy_to_i(x_idx - 1, y_idx, col_size, mesh_size)]
    + a[xy_to_i(x_idx + 1, y_idx, col_size, mesh_size)]
    + a[xy_to_i(x_idx, y_idx - 1, col_size, mesh_size)]
    + a[xy_to_i(x_idx, y_idx + 1, col_size, mesh_size)];
    }
";

In [4]:
sum_kernel = "
   __kernel void sum(__global float *a,
__global const float *diff,
float step_size)
    {
      int gid = get_global_id(0);
    a[gid] += step_size * diff[gid];
    }
";

In [5]:
function initial_cond()
    a = zeros(Float32, (500, 500))
    a[Int(size(a)[1] / 2), Int(size(a)[2] / 2)] = 10
    return a
end

initial_cond (generic function with 1 method)

In [6]:
a = initial_cond();

In [7]:
device, ctx, queue = cl.create_compute_context()

a_buff = cl.Buffer(Float32, ctx, (:rw, :copy), hostbuf=a)
diff_buff = cl.Buffer(Float32, ctx, :rw, length(a))

diff_p = cl.Program(ctx, source=diff_kernel) |> cl.build!
diff_k = cl.Kernel(diff_p, "diff")

sum_p = cl.Program(ctx, source=sum_kernel) |> cl.build!
sum_k = cl.Kernel(sum_p, "sum")

step_size = 0.001

for i in 1:10000
    queue(diff_k, length(a), nothing, a_buff, diff_buff, size(a)[1], length(a))
    queue(sum_k, length(a), nothing, a_buff, diff_buff, Float32(step_size))
end
r = cl.read(queue, a_buff);


In [8]:
r = reshape(r, size(a))

500×500 Array{Float32,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [9]:
#plot(x=1:length(a), y=r, Geom.line)
#spy(r)

In [10]:
a

500×500 Array{Float32,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [11]:
Pkg.add('PyPlot')

LoadError: syntax: invalid character literal